# A_Star

Thuận toán A Star:

Là thuật toán tìm kiếm thông minh, tìm kiếm dựa trên công thức:

> <b>F = G + H</b>

trong đó:
F: là chi phí tính được, thuận toán sẽ dựa trên F nhỏ nhất trong cấc lựa chọn tính ra để quyết định đường đi tiếp theo
G: Chi phí để đi đường đi đó
H: Là chi phí ước lượng từ vị trí tiếp theo so với đích đến

### Các cách để tính H (Heuristics)

<b>I. Heuristic chuẩn xác</b>
- tính giá trị chính xác bằng cách sử dụng chính chi phí đường đi G (nhưng rất tốn thời gian)



<b>II. Heuristic tương đối dành cho bài toán kiểu ô/vị trí tọa độ</b>
1. Manhattan distance
> H = abs(current_cell.x - goal.x) + abs(current_cell.y - goal.y)

điều kiện: Chỉ khi di chuyển sang 4 hướng trái phải lên xuống.



2. Diagonal distance
> dx = abs(current_cell.x - goal.x)<br>
> dy = abs(current_cell.y - goal.y)<br>
> H = D * (dx + dy) + (D2 - 2*D) * min(dx, dy)<br>
> (D usually = 1, D2 usually = sqrt(2) )<br>

điều kiện: Chỉ khi di chuyển cả 8 hướng.



3. Euclidean distance
> H = sqrt((current_cell.x - goal.x)^2 + (current_cell.y - goal.y)^2)

điều kiện: Sử dụng khi có thể di chuyển tự do.



<b>III. Heuristic tương đối dành cho bài toán kiểu dạng không gian</b>

### Setting up
install these libraries

In [17]:
!pip install networkx
!pip install matplotlib

### Bài toán Người du lịch

tìm cách để đi qua được tất cả các thành phố/nodes/vị trí.



In [3]:
import heapq
from math import inf
import networkx as nx
import matplotlib.pyplot as plt

In [9]:
def mst_cost(nodes, graph):
    """Compute MST cost using Prim's algorithm for given nodes."""
    if not nodes:
        return 0
    visited = set()
    total_cost = 0
    start = next(iter(nodes))
    visited.add(start)

    while len(visited) < len(nodes):
        min_edge = inf
        next_node = None
        for u in visited:
            for v, cost in graph[u]:
                if v in nodes and v not in visited and cost < min_edge:
                    min_edge = cost
                    next_node = v
        if next_node:
            visited.add(next_node)
            total_cost += min_edge
        else:
            break
    return total_cost
    

In [16]:
def tsp_a_star(graph, start):
    cities = list(graph.keys())
    start_city = start

    # Priority Queue: (f, g, current_city, visited_tuple, path)
    pq = []
    heapq.heappush(pq, (0, 0, start_city, frozenset([start_city]), [start_city]))

    best_cost = inf
    best_path = None
    step = 0

    
    while pq:
        f, g, current, visited, path = heapq.heappop(pq)

        print(f"\n=== Step {step} ===")
        print(f"Expanding Node: {current}")
        print(f"Path so far: {path}")
        print(f"Cost so far (g): {g}, Estimated total cost (f): {f}")
        print(f"Visited: {visited}")
        step += 1

        # Goal: visited all cities and return to start
        if len(visited) == len(cities):
            return_cost = next(cost for v, cost in graph[current] if v == start_city)
            total_cost = g + return_cost
            print(f"Reached goal! Returning to start costs {return_cost}, Total: {total_cost}")
            if total_cost < best_cost:
                best_cost = total_cost
                best_path = path + [start_city]
            continue

        # Generate successors
        unvisited = set(cities) - visited
        print(f"Unvisited cities: {unvisited}")

        for neighbor, cost in graph[current]:
            if neighbor not in visited:
                g_new = g + cost

                # Heuristic calculation:
                nearest_unvisited_from_current = min(
                    (c for c in (cost for v, cost in graph[current] if v in unvisited)), default=0
                )
                mst_for_unvisited = mst_cost(unvisited, graph)
                nearest_to_start = min(
                    (cost for v in unvisited for nv, cost in graph[v] if nv == start_city),
                    default=0
                )

                h_new = nearest_unvisited_from_current + mst_for_unvisited + nearest_to_start
                f_new = g_new + h_new

                print(f"\n  Neighbor: {neighbor}")
                print(f"    Edge cost: {cost}")
                print(f"    g_new: {g_new}")
                print(f"    Heuristic breakdown:")
                print(f"      Nearest from current: {nearest_unvisited_from_current}")
                print(f"      MST for unvisited: {mst_for_unvisited}")
                print(f"      Nearest back to start: {nearest_to_start}")
                print(f"    h_new: {h_new}, f_new: {f_new}")

                new_visited = visited | {neighbor}
                heapq.heappush(pq, (f_new, g_new, neighbor, frozenset(new_visited), path + [neighbor]))

        print(f"Priority queue size: {len(pq)}")
    
    print("\n=== Search Complete ===")
    return best_path, best_cost
    

### Sample 1

In [17]:
graph = {
    'A': [('B', 2), ('C', 9), ('D', 10), ('E', 3)],
    'B': [('A', 2), ('C', 6), ('D', 4), ('E', 3)],
    'C': [('A', 9), ('B', 6), ('D', 8), ('E', 5)],
    'D': [('A', 10), ('B', 4), ('C', 8), ('E', 1)],
    'E': [('A', 3), ('B', 3), ('C', 5), ('D', 1)],
}

path, cost = tsp_a_star(graph, 'A')
print("Best Path:", path)
print("Minimum Cost:", cost)


=== Step 0 ===
Expanding Node: A
Path so far: ['A']
Cost so far (g): 0, Estimated total cost (f): 0
Visited: frozenset({'A'})
Unvisited cities: {'C', 'D', 'E', 'B'}

  Neighbor: B
    Edge cost: 2
    g_new: 2
    Heuristic breakdown:
      Nearest from current: 2
      MST for unvisited: 9
      Nearest back to start: 2
    h_new: 13, f_new: 15

  Neighbor: C
    Edge cost: 9
    g_new: 9
    Heuristic breakdown:
      Nearest from current: 2
      MST for unvisited: 9
      Nearest back to start: 2
    h_new: 13, f_new: 22

  Neighbor: D
    Edge cost: 10
    g_new: 10
    Heuristic breakdown:
      Nearest from current: 2
      MST for unvisited: 9
      Nearest back to start: 2
    h_new: 13, f_new: 23

  Neighbor: E
    Edge cost: 3
    g_new: 3
    Heuristic breakdown:
      Nearest from current: 2
      MST for unvisited: 9
      Nearest back to start: 2
    h_new: 13, f_new: 16
Priority queue size: 4

=== Step 1 ===
Expanding Node: B
Path so far: ['A', 'B']
Cost so far (g): 2,

### Full code for TSP problem (no explanation)

In [7]:
import heapq
from math import inf

def mst_cost(nodes, graph):
    """Compute MST cost using Prim's algorithm for given nodes."""
    if not nodes:
        return 0
    visited = set()
    total_cost = 0
    start = next(iter(nodes))
    visited.add(start)

    while len(visited) < len(nodes):
        min_edge = inf
        next_node = None
        for u in visited:
            for v, cost in graph[u]:
                if v in nodes and v not in visited and cost < min_edge:
                    min_edge = cost
                    next_node = v
        if next_node:
            visited.add(next_node)
            total_cost += min_edge
        else:
            break
    return total_cost

def tsp_a_star(graph, start):
    cities = list(graph.keys())
    start_city = start

    # Priority Queue: (f, g, current_city, visited_tuple, path)
    pq = []
    heapq.heappush(pq, (0, 0, start_city, frozenset([start_city]), [start_city]))

    best_cost = inf
    best_path = None

    while pq:
        f, g, current, visited, path = heapq.heappop(pq)

        # Goal: visited all cities and return to start
        if len(visited) == len(cities):
            return_cost = next(cost for v, cost in graph[current] if v == start_city)
            total_cost = g + return_cost
            if total_cost < best_cost:
                best_cost = total_cost
                best_path = path + [start_city]
            continue

        # Generate successors
        unvisited = set(cities) - visited
        for neighbor, cost in graph[current]:
            if neighbor not in visited:
                g_new = g + cost

                # Heuristic calculation:
                nearest_unvisited_from_current = min(
                    (c for c in (cost for v, cost in graph[current] if v in unvisited)), default=0
                )
                mst_for_unvisited = mst_cost(unvisited, graph)
                nearest_to_start = min(
                    (cost for v in unvisited for nv, cost in graph[v] if nv == start_city),
                    default=0
                )

                h_new = nearest_unvisited_from_current + mst_for_unvisited + nearest_to_start
                f_new = g_new + h_new

                new_visited = visited | {neighbor}
                heapq.heappush(pq, (f_new, g_new, neighbor, frozenset(new_visited), path + [neighbor]))

    return best_path, best_cost

In [8]:
graph = {
    'A': [('B', 2), ('C', 9), ('D', 10), ('E', 3)],
    'B': [('A', 2), ('C', 6), ('D', 4), ('E', 3)],
    'C': [('A', 9), ('B', 6), ('D', 8), ('E', 5)],
    'D': [('A', 10), ('B', 4), ('C', 8), ('E', 1)],
    'E': [('A', 3), ('B', 3), ('C', 5), ('D', 1)],
}

path, cost = tsp_a_star(graph, 'A')
print("Best Path:", path)
print("Minimum Cost:", cost)

Best Path: ['A', 'B', 'C', 'D', 'E', 'A']
Minimum Cost: 20
